<a href="https://colab.research.google.com/github/Hans-HDA/publiek_domein/blob/main/regen_analyse/schiphol_regen_analyse_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌧️ Analyse regenval Schiphol

**KNMI Station 240 - Regenval afgelopen 50 jaar**

Dit notebook analyseert de regendata van Schiphol als benadering voor de regenval in Bloemendaal om trends in regen te herkennen.

---

## 📋 Stappen:
1. Upload het Excel bestand met Schiphol data
2. Verwerk de data met het onderstaande script
3. Bestudeer de tabel met de regenval gedurende cohorts van 10 jaar  (1975-2024)

---

## 1️⃣ Setup & Dependencies

In [ ]:
# Install required packages
!pip install pandas openpyxl matplotlib seaborn -q

print("✅ Packages geïnstalleerd!")

✅ Packages geïnstalleerd!


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("📚 Libraries geïmporteerd!")
print(f"🐼 Pandas versie: {pd.__version__}")

📚 Libraries geïmporteerd!
🐼 Pandas versie: 2.2.2


## 2️⃣ Upload Excel Bestand

**Upload het Schiphol Excel bestand met minimaal de kolommen:**
- `STN` - Station nummer (240)
- `YYYYMMDD` - Datum in formaat YYYYMMDD
- `RH` - Neerslag in 0.1mm
- `RHX` - Max uurlijkse neerslag in 0.1mm

Andere weer stations kunnen ook ge-analyseerd worden in dit Notebook.

In [ ]:
from google.colab import files

print("📁 Upload het Excel bestand:")
uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]
print(f"✅ Bestand geüpload: {filename}")

📁 Upload je Excel bestand:


Saving regen_schiphol.xlsx to regen_schiphol.xlsx
✅ Bestand geüpload: regen_schiphol.xlsx


## 3️⃣ Data Processing

In [ ]:
def load_schiphol_data(filepath):
    """
    Load en process Schiphol rainfall data
    """
    print("📊 Excel bestand wordt geladen...")

    # Probeer verschillende sheet namen
    try:
        # Eerst proberen met automatische sheet detectie
        df = pd.read_excel(filepath)
    except:
        try:
            # Probeer specifieke sheet naam
            df = pd.read_excel(filepath, sheet_name='etmgeg_240')
        except:
            # Probeer eerste sheet
            df = pd.read_excel(filepath, sheet_name=0)

    print(f"📋 Oorspronkelijke data: {len(df):,} rijen, {len(df.columns)} kolommen")
    print(f"📅 Kolommen: {list(df.columns)}")

    # Filter missing data (RH = -1 betekent geen data)
    df_clean = df[df['RH'] >= 0].copy()
    print(f"🧹 Na filtering: {len(df_clean):,} rijen")

    # Convert rainfall from 0.1mm to mm
    df_clean['rainfall_mm'] = df_clean['RH'] / 10
    df_clean['max_hourly_mm'] = df_clean['RHX'] / 10

    # Parse date
    df_clean['date'] = pd.to_datetime(df_clean['YYYYMMDD'], format='%Y%m%d')
    df_clean['year'] = df_clean['date'].dt.year
    df_clean['month'] = df_clean['date'].dt.month

    # Exclude 2025 (incomplete year)
    df_clean = df_clean[df_clean['year'] <= 2024]

    print(f"📊 Finale dataset: {len(df_clean):,} dagen")
    print(f"📅 Periode: {df_clean['year'].min()} - {df_clean['year'].max()}")

    return df_clean

# Load the data
df = load_schiphol_data(filename)

📊 Excel bestand wordt geladen...
📋 Oorspronkelijke data: 19,946 rijen, 4 kolommen
📅 Kolommen: ['STN', 'YYYYMMDD', 'RH', 'RHX']
🧹 Na filtering: 17,258 rijen
📊 Finale dataset: 17,069 dagen
📅 Periode: 1971 - 2024


In [ ]:
# Quick data overview
print("🔍 DATA OVERZICHT")
print("=" * 50)

total_days = len(df)
rainy_days = len(df[df['rainfall_mm'] > 0])
dry_days = len(df[df['rainfall_mm'] == 0])

print(f"📊 Totaal dagen: {total_days:,}")
print(f"🌧️  Regendagen: {rainy_days:,} ({rainy_days/total_days*100:.1f}%)")
print(f"☀️  Droge dagen: {dry_days:,} ({dry_days/total_days*100:.1f}%)")

avg_rain_per_rainy_day = df[df['rainfall_mm'] > 0]['rainfall_mm'].mean()
print(f"💧 Gemiddelde regen per regendag: {avg_rain_per_rainy_day:.1f} mm")

print("\n📈 Eerste 5 rijen:")
display(df[['date', 'rainfall_mm', 'max_hourly_mm']].head())

🔍 DATA OVERZICHT
📊 Totaal dagen: 17,069
🌧️  Regendagen: 10,097 (59.2%)
☀️  Droge dagen: 6,972 (40.8%)
💧 Gemiddelde regen per regendag: 4.4 mm

📈 Eerste 5 rijen:


,date,rainfall_mm,max_hourly_mm
1,1971-01-02,0.2,0.1
5,1971-01-06,0.0,0.0
6,1971-01-07,1.9,1.2
7,1971-01-08,1.2,0.7
8,1971-01-09,0.0,0.0


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['rainfall_mm'].plot(kind='hist', bins=20, title='rainfall_mm')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['max_hourly_mm'].plot(kind='hist', bins=20, title='max_hourly_mm')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'date'}, axis=1)
              .sort_values('date', ascending=True))
  xs = counted['date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['rainfall_mm']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_8.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('rainfall_mm')

from matplotlib import pyplot as plt
_df_9['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['rainfall_mm'].plot(kind='line', figsize=(8, 4), title='rainfall_mm')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['max_hourly_mm'].plot(kind='line', figsize=(8, 4), title='max_hourly_mm')
plt.gca().spines[['top', 'right']].set_visible(False)

## 4️⃣ Cohort Analyse: 10-Jaar Perioden

In [ ]:
def create_climate_cohort_table(df):
    """
    Create 10-year climate analysis cohorts (1975-2024)
    """
    # Define 10-year cohorts
    cohort_ranges = {
        '1975-1984': (1975, 1984),
        '1985-1994': (1985, 1994),
        '1995-2004': (1995, 2004),
        '2005-2014': (2005, 2014),
        '2015-2024': (2015, 2024)
    }

    results = []

    for cohort_name, (start_year, end_year) in cohort_ranges.items():
        # Filter data for this cohort
        cohort_data = df[(df['year'] >= start_year) & (df['year'] <= end_year)]

        if len(cohort_data) == 0:
            continue

        # Basic statistics
        total_days = len(cohort_data)
        rainy_days_data = cohort_data[cohort_data['rainfall_mm'] > 0]
        rainy_days = len(rainy_days_data)
        dry_days = total_days - rainy_days

        # Rainfall statistics
        total_rainfall = cohort_data['rainfall_mm'].sum()
        avg_rain_per_rainy_day = rainy_days_data['rainfall_mm'].mean() if rainy_days > 0 else 0
        avg_rain_per_day = cohort_data['rainfall_mm'].mean()

        # Rainfall categories
        light_rain = len(cohort_data[(cohort_data['rainfall_mm'] > 0) & (cohort_data['rainfall_mm'] < 5)])
        moderate_rain = len(cohort_data[(cohort_data['rainfall_mm'] >= 5) & (cohort_data['rainfall_mm'] < 25)])
        heavy_rain = len(cohort_data[(cohort_data['rainfall_mm'] >= 25)])

        # Extreme events percentage
        heavy_days = heavy_rain
        heavy_percentage = (heavy_days / total_days) * 100

        results.append({
            'Decennium': cohort_name,
            'Totaal_Dagen': total_days,
            'Regendagen': rainy_days,
            'Droge_Dagen': dry_days,
            'Procent_Regendagen': round((rainy_days / total_days) * 100, 1),
            'Totale_Neerslag_mm': round(total_rainfall, 0),
            'Gem_per_Dag_mm': round(avg_rain_per_day, 2),
            'Gem_per_Regendag_mm': round(avg_rain_per_rainy_day, 1),
            'Licht_0_5mm': light_rain,
            'Matig_5_25mm': moderate_rain,
            'Zwaar_25mm_en_meer': heavy_rain
            })

    return pd.DataFrame(results)

# Create the cohort analysis table
print("🔄 Cohort analyse wordt uitgevoerd...")
cohort_table = create_climate_cohort_table(df)
print("✅ Cohort analyse voltooid!")

🔄 Cohort analyse wordt uitgevoerd...
✅ Cohort analyse voltooid!


## 🎯 RESULTAAT: Klimaatverandering Tabel

**10-Jaar Cohorten Analyse (1975-2024)**

Deze tabel toont de kerngegevens voor klimaatverandering analyse:

In [ ]:
# Display the main results table
print("🌡️ KLIMAATVERANDERING ANALYSE TABEL")
print("=" * 80)
print("📊 KNMI Station 240 (Schiphol) - Extreme Regengebeurtenissen")
print("📅 Periode: 1975-2024 (2025 uitgesloten - onvolledig jaar)")
print("🎯 Focus: Toename van extreme neerslag door klimaatverandering")
print("=" * 80)

# Display the raw dataframe without styling
display(cohort_table)

🌡️ KLIMAATVERANDERING ANALYSE TABEL
📊 KNMI Station 240 (Schiphol) - Extreme Regengebeurtenissen
📅 Periode: 1975-2024 (2025 uitgesloten - onvolledig jaar)
🎯 Focus: Toename van extreme neerslag door klimaatverandering


,Decennium,Totaal_Dagen,Regendagen,Droge_Dagen,Procent_Regendagen,Totale_Neerslag_mm,Gem_per_Dag_mm,Gem_per_Regendag_mm,Licht_0_5mm,Matig_5_25mm,Zwaar_25mm_en_meer
0,1975-1984,3090,1850,1240,59.9,7271.0,2.35,3.9,1359,480,11
1,1985-1994,3067,1832,1235,59.7,8183.0,2.67,4.5,1272,542,18
2,1995-2004,3140,1873,1267,59.6,8745.0,2.79,4.7,1241,609,23
3,2005-2014,3310,1927,1383,58.2,8371.0,2.53,4.3,1383,516,28
4,2015-2024,3259,1906,1353,58.5,8795.0,2.70,4.6,1305,582,19


In [ ]:
# Climate change analysis
print("\n🔍 BEVINDINGEN ANALYSE")
print("=" * 50)

# Compare first and last decade
baseline = cohort_table.iloc[0]  # 1975-1984
recent = cohort_table.iloc[-1]   # 2015-2024

print(f"📊 Vergelijking: {baseline['Decennium']} vs {recent['Decennium']}")
print()

# Extreme events comparison
baseline_extreme = baseline['Zwaar_25mm_en_meer']
recent_extreme = recent['Zwaar_25mm_en_meer']
extreme_increase = recent_extreme - baseline_extreme
extreme_multiplier = recent_extreme / baseline_extreme if baseline_extreme > 0 else 0
percentage_regen_toename = (extreme_multiplier - 1) * 100
percentage_intensiteit_toename = (recent['Gem_per_Regendag_mm'] - baseline['Gem_per_Regendag_mm']) / baseline['Gem_per_Regendag_mm'] * 100

print(f"🌩️  ZWARE REGENVAL (>25mm):")
print(f"   • {baseline['Decennium']}: {baseline_extreme} dagen")
print(f"   • {recent['Decennium']}: {recent_extreme} dagen")
print(f"   • Toename: +{extreme_increase} dagen ({percentage_regen_toename:.0f}% meer)")
print()

# Average intensity
avg_increase = recent['Gem_per_Regendag_mm'] - baseline['Gem_per_Regendag_mm']
print(f"💧 GEMIDDELDE INTENSITEIT:")
print(f"   • {baseline['Decennium']}: {baseline['Gem_per_Regendag_mm']:.1f} mm/regendag")
print(f"   • {recent['Decennium']}: {recent['Gem_per_Regendag_mm']:.1f} mm/regendag")
print(f"   • Toename: +{avg_increase:.1f} mm/regendag, ({percentage_intensiteit_toename:.0f}% meer)")
print()

# Climate conclusion
print("🌡️ KLIMAAT CONCLUSIE:")
if extreme_multiplier > 1.5:
    print("   ⚠️  DUIDELIJKE TOENAME van zware regenval!")
    print("   📈 Dit is consistent met klimaatverandering bevindingen.")
elif extreme_multiplier > 1.2:
    print("   📊 MATIGE TOENAME van zware regenval.")
    print("   📈 Trend wijst op klimaatverandering.")
else:
    print("   📊 Relatief stabiel patroon in zware regenval.")


print(f"\n💡 Data ondersteunt klimaatverandering argument: {percentage_regen_toename:.0f}% meer dagen met zware regenval!")


🔍 BEVINDINGEN REGENVAL ANALYSE
📊 Vergelijking: 1975-1984 vs 2015-2024

🌩️  ZWARE REGENVAL (>25mm):
   • 1975-1984: 11 dagen
   • 2015-2024: 19 dagen
   • Toename: +8 dagen (73% meer)

💧 GEMIDDELDE INTENSITEIT:
   • 1975-1984: 3.9 mm/regendag
   • 2015-2024: 4.6 mm/regendag
   • Toename: +0.7 mm/regendag, (18% meer)

🌡️ KLIMAAT CONCLUSIE:
   ⚠️  DUIDELIJKE TOENAME van zware regenval!
   📈 Dit is consistent met klimaatverandering bevindingen.

💡 Data ondersteunt klimaatverandering argument: 73% meer dagen met zware regenval!


In [ ]:
# Export options
print("📁 EXPORT OPTIES")
print("=" * 30)

# Save as CSV
csv_filename = 'schiphol_klimaat_cohort_analyse.csv'
cohort_table.to_csv(csv_filename, index=False)
print(f"✅ CSV bestand opgeslagen: {csv_filename}")

# Save as Excel
excel_filename = 'schiphol_klimaat_analyse.xlsx'
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    cohort_table.to_excel(writer, sheet_name='Klimaat_Cohort_Analyse', index=False)

    # Also save raw data summary
    summary_stats = df.groupby('year').agg({
        'rainfall_mm': ['count', 'sum', 'mean', 'max'],
        'date': 'count'
    }).round(2)
    summary_stats.to_excel(writer, sheet_name='Jaarlijkse_Statistieken')

print(f"✅ Excel bestand opgeslagen: {excel_filename}")

# Download files
print("\n⬇️  Download bestanden:")
files.download(csv_filename)
files.download(excel_filename)

📁 EXPORT OPTIES
✅ CSV bestand opgeslagen: schiphol_klimaat_cohort_analyse.csv
✅ Excel bestand opgeslagen: schiphol_klimaat_analyse.xlsx

⬇️  Download bestanden:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>